# Analysis

In [1]:
from pathlib import Path
import warnings

import cmocean as cm
import intake
import matplotlib.pyplot as plt
import xarray as xr

from fluxerror import solubility as sol
from fluxerror import gas_transfer_velocity as kw

from plotting import (
    create_grid,
    add_coastline,
    add_continents,
    add_colorbar,
    add_title,
    plot_data
)

# Analysis

In [ ]:
ds_sst = xr.open_dataset("/home/ljg48/project/data/oae-uncertainty-data/oisst-1x1-1993-2022.nc")
ds_sss = xr.open_dataset("/home/ljg48/project/data/oae-uncertainty-data/en4-1x1-1993-2022.nc")
ds_wind = xr.open_dataset("/home/ljg48/project/data/oae-uncertainty-data/wind-1x1-1993-2022.nc")

In [10]:
temp_C = ds_sst['sst']
S = ds_sss['salinity']

delta_temp_C = ds_sst['err']
delta_S = ds_sss['salinity_uncertainty']

input_kwargs = dict(
    temp_C = ds_sst['sst'],
    S = ds_sss['salinity'],
    delta_T = ds_sst['err'],
    delta_S = ds_sss['salinity_uncertainty'],
    u_mean = ds_wind['ws_mean'].mean("product"),
    u_std = ds_wind['ws_std'].mean("product"),
    delta_umean = ds_wind['ws_mean'].std("product"),
    delta_ustd = ds_wind['ws_std'].std("product")
)

In [ ]:
frac_sol_sss = sol.weiss1974.frac.ko_wrt_salt(**input_kwargs).to_dataset(name="frac_sol_sss")
frac_sol_sst = sol.weiss1974.frac.ko_wrt_temp(**input_kwargs).to_dataset(name="frac_sol_sst")
frac_kw_umean = kw.wanninkhof2014.frac.kw_umean(**input_kwargs).to_dataset(name="frac_kw_umean")
frac_kw_ustd = kw.wanninkhof2014.frac.kw_ustd(**input_kwargs).to_dataset(name="frac_kw_ustd")
frac_kw_sc = kw.wanninkhof2014.frac.kw_sc(**input_kwargs).to_dataset(name="frac_kw_sc")

In [10]:
frac_sol_squared = (frac_sol_sss**2) + (frac_sol_sst**2)